## Contour plot function

In [6]:
%matplotlib qt5
from biorefineries import lipidcane2g as lc
import biosteam as bst
import numpy as np
import pandas as pd
from biosteam.utils import colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from biosteam.utils import colors
from biosteam.plots import plot_contour_2d, MetricBar, plot_scatter_points, plot_contour_single_metric, plot_vertical_line
from math import floor, ceil
from biosteam import plots
from biosteam.utils import CABBI_colors
from thermosteam.units_of_measure import format_units
from biosteam.plots.utils import style_axis, style_plot_limits, fill_plot, set_axes_labels
from biosteam import Metric
from warnings import filterwarnings
filterwarnings('ignore', category=bst.utils.DesignWarning)
    
shadecolor = (*colors.neutral.RGBn, 0.20)
linecolor = (*colors.neutral_shade.RGBn, 0.85)
markercolor = (*colors.orange_tint.RGBn, 1)
edgecolor = (*colors.CABBI_black.RGBn, 1)

def tickmarks(data, accuracy=50, N_points=5):
    dmin = data.min()
    dmax = data.max()
    dmin = floor(dmin/accuracy) * accuracy
    dmax = ceil(dmax/accuracy) * accuracy
    step = (dmax - dmin) / (N_points - 1)
    if step == 0:
        return [0, 1]
    else:
        return [dmin + step * i for i in range(N_points)]
    
CABBI_colors = (colors.CABBI_yellow.tint(75).RGBn, 
                colors.CABBI_yellow.RGBn,
                colors.CABBI_green.RGBn,
                colors.CABBI_teal_green.shade(60).RGBn)

CABBI_colors_x = (colors.CABBI_blue_light.tint(90).RGBn,
                  colors.CABBI_blue_light.tint(40).RGBn, 
                  colors.CABBI_blue_light.RGBn, 
#                   colors.CABBI_teal.RGBn,
#                   colors.CABBI_teal_green.tint(10).RGBn,
                  colors.CABBI_teal_green.tint(40).shade(15).RGBn,
                  colors.CABBI_teal_green.shade(45).RGBn)

colormaps = [
    LinearSegmentedColormap.from_list('CABBI', CABBI_colors, 25),
    LinearSegmentedColormap.from_list('CABBI', CABBI_colors_x, 25),
    plt.cm.get_cmap('inferno_r'),
    plt.cm.get_cmap('copper_r'),
    plt.cm.get_cmap('bone_r'),
]

def create_contour_plots(load=False, metric_index=0):
    # %% Generate contour data
    x = np.linspace(0.3, 1., 10)
    y = np.linspace(0.02, 0.15, 10)
    X, Y = np.meshgrid(x, y)
    dollar_per_mt = format_units(r'\$/MT')
    metric = bst.metric
    kg_per_ton = 907.18474
    
    NG = None
    @metric(units=format_units(r'\$/ton'))
    def MFPP():
        return kg_per_ton * lc.lipidcane_tea.solve_price(lc.lipidcane)

    @metric(units=format_units(r'10^6*\$'))
    def TCI():
        return lc.lipidcane_tea.TCI / 1e6 # 10^6*$
    
    metrics = [MFPP, TCI]
    file = 'lipid_extraction_analysis'
    configurations = [1, 2]
    agile = [False, True]
    if load:
        data = np.load(file + '.npy')
    else:
        data = lc.evaluate_across_configurations(X, Y, 0.85, agile, configurations, metrics)
    np.save(file, data)
    data = data[:, :, :, :, metric_index]
    # %% Plot contours
    xlabel = 'Lipid extraction[%]'
    ylabel = "Lipid content [dry wt. %]"
    xticks = [30, 40, 60, 80, 100]
    yticks = [2, 5, 10, 15]
    metric = metrics[metric_index]
    metric_bar = MetricBar(metric.name, metric.units, colormaps[metric_index], tickmarks(data, 5, 5), 18)
    fig, axes, CSs, CB = plot_contour_single_metric(
        100.*X, 100.*Y, data, xlabel, ylabel, xticks, yticks, metric_bar, 
        fillblack=False, styleaxiskw=dict(xtick0=False), label=True
    )
    M = len(configurations)
    N = len(agile)
    for i in range(M):
        for j in range(N):
            ax = axes[i, j]
            CS = CSs[i, j]
            plt.sca(ax)
            metric_data = data[:, :, i, j]
            lb = metric_data.min()
            ub = metric_data.max()
            levels = [i for i in CS.levels if lb <= i <= ub]
            CS = plt.contour(100.*X, 100.*Y, data=metric_data, zorder=1e16, linestyles='dashed', linewidths=1.,
                             levels=levels, colors=[linecolor])
            ax.clabel(CS, levels=CS.levels, inline=True, fmt=lambda x: f'{round(x):,}',
                      fontsize=10, colors=[linecolor], zorder=1e16)
            if j == 0:
                lb = 47.5
                ub = 52.5
            else:
                lb = 75
                ub = 80
            baseline = (lb + ub) / 2.
            plt.fill_between([lb, ub], [2], [20], 
                             color=shadecolor,
                             linewidth=1)
            plot_vertical_line(lb, ls='-.',
                               color=linecolor,
                               linewidth=1.0)
            plot_vertical_line(ub, ls='-.',
                               color=linecolor,
                               linewidth=1.0)
            plot_scatter_points([baseline], [10], marker='*', s=125, color=markercolor,
                                edgecolor=edgecolor)


    plt.show()

## Contour plots

In [7]:
create_contour_plots(load=True, metric_index=1)

<ipython-input-6-91b01dd7c07c>:107: UserWarning: No contour levels were found within the data range.
  CS = plt.contour(100.*X, 100.*Y, data=metric_data, zorder=1e16, linestyles='dashed', linewidths=1.,


In [ ]:
create_contour_plots(2, load=True)
plt.suptitle('Biodiesel production from stillage, 1g ethanol production from juice, and 2g ethanol production from bagasse')

In [ ]:
create_contour_plots(3, load=True)
plt.suptitle('Biodiesel production from stillage, ethanol production from juice and bagasse (combined)')